In [1]:
import numpy as np
import tensorflow as tf
import os, shutil
import numpy as np
import keras
from google.cloud import aiplatform
from enceladus.jobs import TestingPipeline
import wandb

# tf.compat.v1.disable_eager_execution()

In [2]:
model = keras.models.load_model('/home/cam/Documents/enceladus/notebooks/artifacts/model-vital-bee-206:v19/')

2023-02-11 21:29:13.937389: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-11 21:29:13.984924: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-11 21:29:13.985182: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-11 21:29:13.985979: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [5]:
@tf.function()
def heartfelt_predict(ppg, vpg, apg):
   inputs = {
        'ppg': ppg,
        'vpg': vpg,
        'apg': apg,
   }
   prediction = model(inputs)
   return {'abp': prediction}

signatures = heartfelt_predict.get_concrete_function(
   ppg=tf.TensorSpec([None, 256, 1], dtype=tf.dtypes.float32, name='ppg'),
   vpg=tf.TensorSpec([None, 256, 1], dtype=tf.dtypes.float32, name='vpg'),
   apg=tf.TensorSpec([None, 256, 1], dtype=tf.dtypes.float32, name='apg'),
)

tf.saved_model.save(
    model,
    export_dir='/home/cam/Documents/enceladus/vital-bee-206-serving/',
    signatures=signatures,
)

INFO:tensorflow:Assets written to: /home/cam/Documents/enceladus/vital-bee-206-serving/assets


INFO:tensorflow:Assets written to: /home/cam/Documents/enceladus/vital-bee-206-serving/assets


In [ ]:
x = np.ones((1, 256))
model.predict([x, x, x])

In [ ]:
saver = tf.saved_model.load(
    export_dir='/home/cam/Documents/enceladus/notebooks/artifacts/model-vital-bee-206:v19/',
    tags=[tf.saved_model.SERVING],
)

p = tf.compat.v1.placeholder(tf.float64, [None, 256], 'ppg')
v = tf.compat.v1.placeholder(tf.float64, [None, 256], 'vpg')
a = tf.compat.v1.placeholder(tf.float64, [None, 256], 'apg')
y = tf.compat.v1.placeholder(tf.float64, [None, 256], 'abp')

sess = tf.compat.v1.Session()
sess.run(tf.compat.v1.global_variables_initializer())

ppg_input = tf.compat.v1.saved_model.utils.build_tensor_info(p)
vpg_input = tf.compat.v1.saved_model.utils.build_tensor_info(v)
apg_input = tf.compat.v1.saved_model.utils.build_tensor_info(a)
info_output = tf.compat.v1.saved_model.utils.build_tensor_info(y)

signature = tf.compat.v1.saved_model.signature_def_utils.build_signature_def(
        inputs={'ppg': ppg_input, 'vpg': vpg_input, 'apg': apg_input},
        outputs={'abp': info_output},
        method_name=tf.saved_model.PREDICT_METHOD_NAME
)

saver.add_meta_graph_and_variables(
        sess,
        [tf.saved_model.SERVING],
        signature_def_map={'predict': signature},
)
saver.save()

# Close and clean up
sess.close()
tf.reset_default_graph()

In [ ]:
# Input (2D)
x = np.array([[x1,x2] for x1 in np.linspace(10,20,4) for x2 in np.linspace(-7,-3,3)])

# Output (3D)
f = np.array([[np.sin(np.sum(xx)),np.cos(np.sum(xx)),np.cos(np.sum(xx))**2] for xx in x])

#%% Model

print('**********************************************')
print('TF - save')

# Dimension of input x and output f
d_x = x.shape[-1]
d_f = f.shape[-1]

# Placeholders
x_p = tf.placeholder(tf.float64,[None,d_x],'my_x_p')
f_p = tf.placeholder(tf.float64,[None,d_f],'my_f_p')

# Model
model = x_p
model = tf.layers.dense(model,7,tf.tanh)
model = tf.layers.dense(model,5,tf.tanh)
model = tf.layers.dense(model,d_f,None)
model = tf.identity(model,'my_model')

# Session
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# Evaluate for later check of serving
f_model = sess.run(model,{x_p:x})
folder = 'data'
if not os.path.exists(folder):
    os.mkdir(folder)
np.savetxt('data/x.dat',f_model)
np.savetxt('data/f_model.dat',f_model)

# Save model
folder = 'saved/model/001'
if os.path.exists(folder):
    shutil.rmtree(folder)
    print('Old model deleted')
saver = tf.saved_model.builder.SavedModelBuilder(folder)
############################################
# HOW DO I SET UP THE SIGNATURE CORRECTLY?
############################################
info_input = tf.saved_model.utils.build_tensor_info(x_p)
info_output = tf.saved_model.utils.build_tensor_info(model)
signature = tf.saved_model.signature_def_utils.build_signature_def(
        inputs={'x':info_input}
        ,outputs={'f':info_output}
        ,method_name=tf.saved_model.signature_constants.PREDICT_METHOD_NAME
        )
saver.add_meta_graph_and_variables(
        sess
        ,[tf.saved_model.tag_constants.SERVING]
        ,signature_def_map={'predict':signature}
        ####################################################################
        ### WHAT DO I NEED TO PUT HERE IN ORDER TO CALL THE MODEL LATER ON 
        ### WHILE SERVING WITH DOCKER AND HOW DO I CALL IT IN DOCKER??
        ####################################################################
        )
saver.save()

# Close and clean up
sess.close()
tf.reset_default_graph()

#%% Load in Python and check

print('**********************************************')
print('TF - load in Python')

# Session
sess = tf.Session()

# Load
tf.saved_model.loader.load(
        sess
        ,[tf.saved_model.tag_constants.SERVING]
        ,folder
        )

# Extract operations from graph
graph = tf.get_default_graph()
x_p = graph.get_tensor_by_name('my_x_p:0')
f_p = graph.get_tensor_by_name('my_f_p:0')
model = graph.get_tensor_by_name('my_model:0')

# Evaluate model
f_model2 = sess.run(model,{x_p:x})
print(f_model - f_model2)

# Close and clean up
sess.close()
tf.reset_default_graph()

In [ ]:
ppg, vpg, apg, abp = TestingPipeline(
    data_path='/home/cam/Documents/database_tools/data/data-2022-11-08/mimic3/',
    scaler_path='/home/cam/Documents/database_tools/data/data-2022-11-08/mimic3/scalers_MinMax.pkl',
    scaler_type='MinMax',
    model_path='heartfelt/Enceladus/model-vital-bee-206:v19',
).run()

In [ ]:
GOOGLE_CLOUD_PROJECT = 'heartfelt-0'
GOOGLE_CLOUD_REGION = 'us-central1'
ENDPOINT_ID = 4207052545266286592

# The AI Platform services require regional API endpoints.
client_options = {
    'api_endpoint': GOOGLE_CLOUD_REGION + '-aiplatform.googleapis.com'
    }
# Initialize client that will be used to create and send requests.
client = aiplatform.gapic.PredictionServiceClient(client_options=client_options)

# Set data values for the prediction request.
# Our model expects 4 feature inputs and produces 3 output values for each
# species. Note that the output is logit value rather than probabilities.
# See the model code to understand input / output structure.
instances = [{
    'ppg': [1] * 256,
    'vpg': [1] * 256,
    'apg': [1] * 256,
}]

endpoint = client.endpoint_path(
    project=GOOGLE_CLOUD_PROJECT,
    location=GOOGLE_CLOUD_REGION,
    endpoint=ENDPOINT_ID,
)
# Send a prediction request and get response.
response = client.predict(endpoint=endpoint, instances=instances)

response.predictions[0]